<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод-по-метрике-качества" data-toc-modified-id="Вывод-по-метрике-качества-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод по метрике качества</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Описание проекта 

**Защита персональных данных клиентов**

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Для задачи подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**Описание данных**

* `Признаки`: пол, возраст и зарплата застрахованного, количество членов его семьи;
* `Целевой признак`: количество страховых выплат клиенту за последние 5 лет


**Построим функцию, которая выведет информацию о датасете:**

In [3]:
def my_info(x, y): # аргумент x примет значение сохраненного датасета в переменной;
                   # аргумент y примет описательный характер датасета, по которому будет описываться информация
    display(x.head())
    print(f'Размер {y}: {x.shape}')
    print()
    print(f'Информация о {y}:')
    print()
    display(x.info())
    display(x.describe())
    print(f'Кол-во пропущенных значений в {y}:') 
    display(x.isna().sum())
    print()
    print(f'Кол-во дубликатов в {y}: {x.duplicated().sum()}')
    
my_info(data, 'data')

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Размер data: (5000, 5)

Информация о data:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Кол-во пропущенных значений в data:


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


Кол-во дубликатов в data: 153


Наблюдаем, несоответствующий тип данных в столбце с `Возрастом`. А также есть вещественный тип данных в колонке с `Зарплатами`. Здесь более логично, чем с возрастом. Проверим на уникальные значения

In [4]:
data['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

In [5]:
data['Зарплата'].value_counts().to_frame()

,Зарплата
45800.0,29
37100.0,28
43200.0,27
41500.0,27
46800.0,26
...,...
14300.0,1
62600.0,1
7400.0,1
70000.0,1


In [6]:
data['Зарплата'].unique()

array([49600., 38000., 21000., 41700., 26100., 41000., 39700., 38600.,
       49700., 51700., 36600., 29300., 39500., 55000., 43700., 23300.,
       48900., 33200., 36900., 43500., 36100., 26600., 48700., 40400.,
       38400., 34600., 34800., 36800., 42200., 46300., 30300., 51000.,
       28100., 64800., 30400., 45300., 38300., 49500., 19400., 40200.,
       31700., 69200., 33100., 31600., 34500., 38700., 39600., 42400.,
       34900., 30500., 24200., 49900., 14300., 47000., 44800., 43800.,
       42700., 35400., 57200., 29600., 37400., 48100., 33700., 61800.,
       39400., 15600., 52600., 37600., 52500., 32700., 51600., 60900.,
       41800., 47400., 26500., 45900., 35700., 34300., 26700., 25700.,
       33300., 31100., 31500., 42100., 37300., 42500., 27300., 46800.,
       33500., 44300., 41600., 53900., 40100., 44600., 45000., 32000.,
       38200., 33000., 38500., 51800., 33800., 46400., 43200., 31800.,
       50200., 35100., 30700., 45800., 49300., 42800., 33600., 50300.,
      

Вещественный тип данных только из-за `0`, преобразовываем для удобства в `'int'` 

In [7]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')
data.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28,35700,2,0
4996,0,34,52400,1,0
4997,0,20,33900,2,0
4998,1,22,32700,3,0
4999,1,28,40600,1,0


Также были обнаружены дубликаты:

In [8]:
print(f'Процент дубликатов в датасете: {data.duplicated().mean():.2%}')

Процент дубликатов в датасете: 3.06%


Это небольшой процент дубликатов. Они подразумевают полные совпадения во всех признаках. Их удаления не принесет особого вреда

In [9]:
data = data.drop_duplicates().reset_index(drop=True)
print(f'Процент дубликатов в датасете: {data.duplicated().mean():.0%}', '\n')
data.info()

Процент дубликатов в датасете: 0% 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 189.5 KB


### Вывод

* Были загружены данные;
* Была введена функция для информации о датасете;
* Был выявлен и исправлен некорректный тип данных;
* Удалены дубликаты

## Умножение матриц

Чтобы записать формулу внутри текста, окружим её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.*

Обозначения:

* $X$ — матрица признаков (нулевой столбец состоит из единиц)
* $y$ — вектор целевого признака
* $P$ — матрица, на которую умножаются признаки
* $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

* Смотрим, как параметры линейной регрессии после умножения на обратимую матрицу $P$ будут относиться к параметрам без умножения, то есть исходной формуле

$$
w' = ((XP)^TXP)^{-1}(XP)^Ty 
$$

**Исходя из этого:**

$$
a'=XPw'=XP((XP)^TXP)^{-1}(XP)^Ty
$$

**Далее для раскрытия скобок понадобится следующие формулы:**


$
AA^{-1}=A^{-1}A=E
$

$
AE=EA=A
$

$
(AB)^{-1}=B^{-1}A^{-1}
$

$
(AB)^T=B^TA^T
$

**Продолжаем раскрывать скобки:**

$$
a'=XP(P^TX^TXP)^{-1}P^TX^Ty=XPP^{-1}(P^TX^TX)^{-1}P^TX^Ty=XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty=XE(X^TX)^{-1}EX^Ty=X(X^TX)^{-1}X^Ty=Xw=a
$$

**Ответ:** Не изменится. Качество Линейной Регрессии не изменится.

* доказали, что параметры линейной регрессии после умножения на обратимую матрицу $P$ равны параметрам без умножения, то есть исходной формуле (в исходной задаче и в преобразованной)

**Доказано**

**Поэтапное решение:**

* После транспонирования переменные под $^{-1}$ степенью образуют три элемента под $^{-1}$ степенью:

  * как $(ABC)$ из формулы
  $$(ABC)^{-1} = C^{-1}(AB)^{-1} = (BC)^{-1}A^{-1} = C^{-1}B^{-1}A^{-1}$$

$$
(P^TX^TXP)^{-1} = (ABC)
$$

  * и получаем
  
$$
P^{-1}(X^TX)^{-1}(P^T)^{-1}
$$

* Затем раскрываем скобки уже в этих переменных, где тоже два элемента. Исходя из правил их раскрытия также меняем местами их;
* После раскрытий у нас получились обратные квадратные матрицы $P$ и $P^{-1}$, $(P^T)^{-1}$ и $P^T$, произведение которых дает единичные матрицы $E$;
* И как итог привод к исходной формуле

## Алгоритм преобразования

* Так как мы показали, что качество Линейной Регрессии при умножении на обратимую матрицу не изменится, то мы создаем рандомную квадратную матрицу (соизмеримо с размером кол-ва созданных в последствии признаков **features**) функцией **np.random.normal()**;
* После проверяем ее (рандомную матрицу) на обратимость функцией **numpy.linalg.inv()** (если матрица необратима, будет обнаружена ошибка. Такая вероятность есть, но получить ее близка к нулю);
* И после проверки умножаем признаки **features** на эту матрицу
* В исходное состояние мы вернем путем умножения новой полученной матрицы на обратимую (т.е. в минус первой степени $P^{-1}$)

## Проверка алгоритма

In [10]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


* Создаем признаки и целевой признак

In [11]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

* Методом **np.random.normal()** с параметром `size` задаем размер равный выделенным признакам сгенерированной случайной матрицы, создавая квадратичную матрицу

In [12]:
random_matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
random_matrix

array([[ 0.8517366 ,  2.58778273,  0.41469525,  0.1924967 ],
       [-0.57550538, -0.59679153, -0.29817632, -1.54209888],
       [ 1.18195647,  0.57741981, -0.48899691, -0.57945818],
       [ 0.66532256,  1.5642632 ,  0.36249899, -0.253462  ]])

* Проверяем и сохраняем в переменную **обратимую матрицу**, чтобы убедиться, что при скалярном умножении признаки можно вернуть в исходную позицию

In [13]:
random_matrix_reverse = np.linalg.inv(random_matrix)
random_matrix_reverse

array([[-1.23803889, -0.61827925,  0.52644876,  1.61789263],
       [ 1.11546995,  0.33063918, -0.04411761, -1.06362973],
       [-2.21989033, -0.59333483, -0.7939499 ,  3.73910163],
       [ 0.45957738, -0.4309589 , -0.02587903, -0.91515031]])

* Проверили на обратимость и выявили для случайной матрицы обратную **random_matrix_reverse;**
* Сохраняем преобразованные признаки **features_converted** путем скалярного умножения признаков **features** и случайно обратимой матрицы **random_matrix**

In [14]:
features_converted = features @ random_matrix
features_converted

,0,1,2,3
0,58602.962492,28619.706157,-24265.694642,-28804.412724
1,44888.538120,21916.064623,-18595.236093,-22090.600833
2,24804.396316,12108.509050,-10277.582169,-12213.342638
3,49276.830034,24069.001971,-20396.707743,-24196.297087
4,30833.801580,15056.534654,-12770.753525,-15166.844758
...,...,...,...,...
4842,42181.062648,20600.305572,-17464.813533,-20730.342702
4843,61915.617423,30238.071382,-25633.213444,-30416.293432
4844,40058.145036,19565.724246,-16582.233689,-19674.981188
4845,38640.163314,18875.778937,-15995.256559,-18982.776536


In [15]:
checking = features_converted @ random_matrix_reverse # проверяем на возвращение к исходной
display(checking)
features

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-7.454312e-12,46.0,38000.0,1.000000e+00
2,-1.735419e-12,29.0,21000.0,-3.607275e-12
3,-1.836763e-11,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,4.220439e-12
...,...,...,...,...
4842,-1.011423e-11,28.0,35700.0,2.000000e+00
4843,-1.953706e-11,34.0,52400.0,1.000000e+00
4844,-1.370779e-11,20.0,33900.0,2.000000e+00
4845,1.000000e+00,22.0,32700.0,3.000000e+00


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4842,0,28,35700,2
4843,0,34,52400,1
4844,0,20,33900,2
4845,1,22,32700,3


* Теперь разделяем признаки на выборки **(обучающую и тестовую)**

In [16]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, 
                                                                            random_state=12345)

display(features_train.shape)
features_test.shape

(3635, 4)

(1212, 4)

* Чтобы проверить, что качество Линейной Регрессии не отличается до и после преобразования, создаем выборки отдельно из преобразованных признаков, разделив методом **train_test_split()** уже на обучающую и тестовую **features_converted**;
* Но модель все также будет обучаться на целевых признаках исходного датасета, разделенного ранее на обучающую и тестовую выборки

In [17]:
features_train_converted, features_test_converted = train_test_split(features_converted, test_size=0.25, 
                                                                     random_state=12345)

* Теперь обучаем модель на исходных и преобразованных данных для вычисления метрики **r2_score**

In [18]:
def my_func(features, target, features_t, target_t, name): # 4 аргумента служат выборками (обучающей/тестововй), которые 
                                                           # мы ставим для соответсвующих данных (исходных, преобразованных)
    model = LinearRegression()                             # аргумент name носит описательный характер для какого датасета
    model.fit(features, target)                            #функция принимает выборки,а возвраащет метрику качества r2_score
    predictions = model.predict(features_t)
    
    return f'r2_score {name}: {r2_score(target_t, predictions)}'

In [19]:
my_func(features_train, target_train, features_test, target_test, 'исходного датасета')

'r2_score исходного датасета: 0.42307727615837565'

In [20]:
my_func(features_train_converted, target_train, features_test_converted, target_test, 'преобразованного датасета')

'r2_score преобразованного датасета: 0.42307727615838486'

### Вывод по метрике качества

* Наблюдаем, что метрики **r2_score** на исходных и на преобразованных данных почти не отличаются
  * **r2_score исходного датасета:** `0.42307727615837565`;
  * **r2_score преобразованного датасета:** `0.42307727615838486`

## Вывод

* Были загружены и разобраны данные;
* Были выявлены и исправлены не совсем подходящие типы данных в колонках `'Возраст'` и `'Зарплата'`;
* Обнаружены были дубликаты. Путем подсчета их процента **3.06%**, и малой вероятности полного совпадения значений признаков, было принято решение их удалить;
* Ответили на вопрос, который гласит: **"Изменится ли параметры линейной регрессии после умножения на обратимую матрицу"**
  * **Ответ: `Не изменится`.** Качество Линейной Регрессии не изменится. Разобрав формулу как аргумент
  
$$
a'=XP(P^TX^TXP)^{-1}P^TX^Ty=XPP^{-1}(P^TX^TX)^{-1}P^TX^Ty=XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty=XE(X^TX)^{-1}EX^Ty=X(X^TX)^{-1}X^Ty=Xw=a
$$

* Был предложен алгоритм решения для задачи **защиты персональных данных клиентов**. Приведен метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию;
* Выполнив алгоритм решения пришли к выводу, что `r2_score на исходных и на преобразованных данных почти не отличаются`, что свидетельствует о разработанном алгоритме **для защиты персональных данных клиентов страховой компании "Хоть потоп"**